In [2]:
!pip install --upgrade numpy pandas openpyxl

In [3]:
pip install openpyxl

In [4]:
import pandas as pd
ground_df = pd.read_csv("Training_data_uhi_index_2025-02-18.csv")
ground_df.head()

,Longitude,Latitude,datetime,UHI Index
0,-73.909167,40.813107,24/07/2021 15:53,1.030289
1,-73.909187,40.813045,24/07/2021 15:53,1.030289
2,-73.909215,40.812978,24/07/2021 15:53,1.023798
3,-73.909242,40.812908,24/07/2021 15:53,1.023798
4,-73.909257,40.812845,24/07/2021 15:53,1.021634


In [5]:
ground_df["datetime"].unique()

array(['24/07/2021 15:53', '24/07/2021 15:54', '24/07/2021 15:55',
       '24/07/2021 15:56', '24/07/2021 15:57', '24/07/2021 15:58',
       '24/07/2021 15:59', '24/07/2021 15:01', '24/07/2021 15:02',
       '24/07/2021 15:03', '24/07/2021 15:04', '24/07/2021 15:05',
       '24/07/2021 15:06', '24/07/2021 15:07', '24/07/2021 15:08',
       '24/07/2021 15:09', '24/07/2021 15:11', '24/07/2021 15:12',
       '24/07/2021 15:13', '24/07/2021 15:14', '24/07/2021 15:15',
       '24/07/2021 15:16', '24/07/2021 15:17', '24/07/2021 15:18',
       '24/07/2021 15:19', '24/07/2021 15:20', '24/07/2021 15:21',
       '24/07/2021 15:22', '24/07/2021 15:23', '24/07/2021 15:24',
       '24/07/2021 15:25', '24/07/2021 15:26', '24/07/2021 15:27',
       '24/07/2021 15:28', '24/07/2021 15:29', '24/07/2021 15:30',
       '24/07/2021 15:31', '24/07/2021 15:32', '24/07/2021 15:33',
       '24/07/2021 15:34', '24/07/2021 15:35', '24/07/2021 15:36',
       '24/07/2021 15:37', '24/07/2021 15:38', '24/07/2021 15:

In [6]:
duplicate_rows = ground_df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_rows}")


Number of duplicate rows: 0


In [7]:
import pandas as pd

# Define function to classify borough
def classify_location(lat, lon):
    if 40.70 <= lat <= 40.88 and -74.03 <= lon <= -73.91:
        return "Manhattan"
    elif 40.78 <= lat <= 40.92 and -73.93 <= lon <= -73.76:
        return "Bronx"
    else:
        return "Unknown"

# Apply function to dataset
ground_df['location'] = ground_df.apply(lambda row: classify_location(row['Latitude'], row['Longitude']), axis=1)

ground_df.head()


,Longitude,Latitude,datetime,UHI Index,location
0,-73.909167,40.813107,24/07/2021 15:53,1.030289,Bronx
1,-73.909187,40.813045,24/07/2021 15:53,1.030289,Bronx
2,-73.909215,40.812978,24/07/2021 15:53,1.023798,Bronx
3,-73.909242,40.812908,24/07/2021 15:53,1.023798,Bronx
4,-73.909257,40.812845,24/07/2021 15:53,1.021634,Bronx


In [8]:
# Ensure file path is correct
bronx_weather = pd.read_excel("NY_Mesonet_Weather.xlsx", sheet_name="Bronx", engine="openpyxl")
manhattan_weather = pd.read_excel("NY_Mesonet_Weather.xlsx", sheet_name="Manhattan", engine="openpyxl")

In [9]:
# Convert datetime columns in both datasets
ground_df["datetime"] = pd.to_datetime(ground_df["datetime"], format="%d/%m/%Y %H:%M")
bronx_weather["Date / Time"] = pd.to_datetime(bronx_weather["Date / Time"])
manhattan_weather["Date / Time"] = pd.to_datetime(manhattan_weather["Date / Time"])

C:\Users\Nguyen Thu Uyen\AppData\Local\Temp\ipykernel_14836\531226821.py:3: FutureWarning: Parsed string "2021-07-24 06:00:00 EDT" included an un-recognized timezone "EDT". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  bronx_weather["Date / Time"] = pd.to_datetime(bronx_weather["Date / Time"])
C:\Users\Nguyen Thu Uyen\AppData\Local\Temp\ipykernel_14836\531226821.py:4: FutureWarning: Parsed string "2021-07-24 06:00:00 EDT" included an un-recognized timezone "EDT". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  manhattan_weather["Date / Time"] = pd.to_datetime(manhattan_weather["Date / Time"])


In [10]:
# Rename the column to 'datetime'
bronx_weather = bronx_weather.rename(columns={"Date / Time": "datetime"})
manhattan_weather = manhattan_weather.rename(columns={"Date / Time": "datetime"})

In [11]:
# Sort datasets by datetime
ground_df = ground_df.sort_values("datetime")
bronx_weather = bronx_weather.sort_values("datetime")
manhattan_weather = manhattan_weather.sort_values("datetime")

In [12]:
# Merge Bronx data (nearest datetime)
bronx_merged = pd.merge_asof(
    ground_df[ground_df["location"] == "Bronx"],
    bronx_weather,
    on="datetime",
    direction="nearest"
)

In [13]:
bronx_merged.head()

,Longitude,Latitude,datetime,UHI Index,location,Air Temp at Surface [degC],Relative Humidity [percent],Avg Wind Speed [m/s],Wind Direction [degrees],Solar Flux [W/m^2]
0,-73.909962,40.803688,2021-07-24 15:01:00,0.998554,Bronx,28.0,40.3,3.0,75,725
1,-73.909753,40.803593,2021-07-24 15:01:00,0.998554,Bronx,28.0,40.3,3.0,75,725
2,-73.908127,40.802668,2021-07-24 15:01:00,0.989779,Bronx,28.0,40.3,3.0,75,725
3,-73.908203,40.802713,2021-07-24 15:01:00,0.991943,Bronx,28.0,40.3,3.0,75,725
4,-73.908290,40.802758,2021-07-24 15:01:00,0.992063,Bronx,28.0,40.3,3.0,75,725


In [14]:
# Merge Manhattan data (nearest datetime)
manhattan_merged = pd.merge_asof(
    ground_df[ground_df["location"] == "Manhattan"],
    manhattan_weather,
    on="datetime",
    direction="nearest"
)

In [15]:
manhattan_merged.head()

,Longitude,Latitude,datetime,UHI Index,location,Air Temp at Surface [degC],Relative Humidity [percent],Avg Wind Speed [m/s],Wind Direction [degrees],Solar Flux [W/m^2]
0,-73.977157,40.771555,2021-07-24 15:01:00,0.975234,Manhattan,26.1,51.1,4.1,139,140
1,-73.910167,40.803775,2021-07-24 15:01:00,1.000718,Manhattan,26.1,51.1,4.1,139,140
2,-73.910268,40.803817,2021-07-24 15:01:00,1.000718,Manhattan,26.1,51.1,4.1,139,140
3,-73.978742,40.771675,2021-07-24 15:01:00,0.975234,Manhattan,26.1,51.1,4.1,139,140
4,-73.978453,40.771677,2021-07-24 15:01:00,0.970907,Manhattan,26.1,51.1,4.1,139,140


In [16]:
# Combine both merged datasets
merged_df = pd.concat([bronx_merged, manhattan_merged])

# Reset index
merged_df.reset_index(drop=True, inplace=True)

# Display the result
merged_df.head()

,Longitude,Latitude,datetime,UHI Index,location,Air Temp at Surface [degC],Relative Humidity [percent],Avg Wind Speed [m/s],Wind Direction [degrees],Solar Flux [W/m^2]
0,-73.909962,40.803688,2021-07-24 15:01:00,0.998554,Bronx,28.0,40.3,3.0,75,725
1,-73.909753,40.803593,2021-07-24 15:01:00,0.998554,Bronx,28.0,40.3,3.0,75,725
2,-73.908127,40.802668,2021-07-24 15:01:00,0.989779,Bronx,28.0,40.3,3.0,75,725
3,-73.908203,40.802713,2021-07-24 15:01:00,0.991943,Bronx,28.0,40.3,3.0,75,725
4,-73.908290,40.802758,2021-07-24 15:01:00,0.992063,Bronx,28.0,40.3,3.0,75,725


In [17]:
merged_df.shape

(11229, 10)

In [18]:
merged_df["datetime"].unique()

<DatetimeArray>
['2021-07-24 15:01:00', '2021-07-24 15:07:00', '2021-07-24 15:08:00',
 '2021-07-24 15:09:00', '2021-07-24 15:10:00', '2021-07-24 15:11:00',
 '2021-07-24 15:12:00', '2021-07-24 15:13:00', '2021-07-24 15:14:00',
 '2021-07-24 15:15:00', '2021-07-24 15:16:00', '2021-07-24 15:17:00',
 '2021-07-24 15:18:00', '2021-07-24 15:19:00', '2021-07-24 15:20:00',
 '2021-07-24 15:21:00', '2021-07-24 15:22:00', '2021-07-24 15:23:00',
 '2021-07-24 15:24:00', '2021-07-24 15:26:00', '2021-07-24 15:27:00',
 '2021-07-24 15:28:00', '2021-07-24 15:29:00', '2021-07-24 15:30:00',
 '2021-07-24 15:31:00', '2021-07-24 15:32:00', '2021-07-24 15:33:00',
 '2021-07-24 15:34:00', '2021-07-24 15:35:00', '2021-07-24 15:36:00',
 '2021-07-24 15:37:00', '2021-07-24 15:38:00', '2021-07-24 15:39:00',
 '2021-07-24 15:40:00', '2021-07-24 15:42:00', '2021-07-24 15:43:00',
 '2021-07-24 15:44:00', '2021-07-24 15:45:00', '2021-07-24 15:46:00',
 '2021-07-24 15:47:00', '2021-07-24 15:48:00', '2021-07-24 15:49:00',
 '20